#  CSCI 3287  -- Final Project
    
## FDA Food Database Exploration and Application

Name: Jarryd Allison

Email: allisonj@colorado.edu

GitHub ID: jarrydallison

In [1]:
import sqlite3
import csv
import os

In [2]:
# Remove the food.db file before running the rest of this notebook
try:
    os.remove('food.db')
except:
    print("food.db doesn't exist yet, skipping")

### Background
The rapid and pervasive onset of obesity in America is confusing given the massive amount of
data available to make wise food choices. Many food items used when grocery shopping are
not necessarily required to have nutrition information, to include things like fresh produce, and
the combination of various food items could yield important information, such as aggregated
nutritional values of a shopping list. The data set and queries involved in this product would be
available to consumers to up ideally in a mobile application, but for the purposes of this project
we will assume that users generally know SQL, or, given enough time, can interact with this
database using a rudimentary frontend such as a Jupyter Notebook to query various food items and
return information about an individual or multiple items.

<hr>

## Food Database Tables

This project uses data from the FDA Food Data Central, a free database based on FDA research that allows users to access a large amount of nutrient information. To begin, we will create five tables from the downloaded csv's contained within this project. You can download various CSV versions of the database on the [FDA Website](https://fdc.nal.usda.gov/download-datasets.html)

<hr>

To fill the tables, we will define a few functions and constants in order to streamline the fills and save space later on in the project.

In [3]:
# The file for our database
database = 'food.db'
# The local directory where our csvs are stored
csvDirectory = 'FoodData_Central_foundation_food_csv_2023-10-26'

# Function to read data from the supplied CSV filename
def readCSV(file):
  with open(file) as f:
    reader = csv.reader(f)
    data = list(reader)
  return data

# Function to create the tables. The individual creation statements
# are supplied as strings to this function.
def generateDB(createTable: str, csvFileName: str, insertTable: str):
  # Create the initial database connection
  conn = sqlite3.connect(database)
  # Create a cursor object
  c = conn.cursor()
  c.execute('PRAGMA foreign_keys=on;')
  # Create the database
  c.execute(createTable)

  # Fill the database
  data = readCSV(csvDirectory+"/"+csvFileName)
  count = 0
  for row in data:
    # Omit the table headers from each CSV
    if (count > 0):
      c.execute(insertTable, row)
    count += 1
  # Commit the changes
  conn.commit()
  conn.close()

# Function to run a query
def runQuery(query: str):
  conn = sqlite3.connect(database)
  c = conn.cursor()
  c.execute('PRAGMA foreign_keys=on;')
  result = c.execute(query).fetchall()
  conn.commit()
  conn.close()
  return result

# Function to drop a table
def dropTable(tableName: str):
  conn = sqlite3.connect(database)
  c = conn.cursor()
  c.execute('PRAGMA foreign_keys=on;')
  result = c.execute('DROP TABLE IF EXISTS '+tableName+';')
  conn.commit()
  conn.close()
    

<hr>

### The Food Table

This is our base table, created from the food.csv available at the [FDA Website](https://fdc.nal.usda.gov/download-datasets.html). Our tables are currently stored in csv form, so we will use python to convert the csv's into SQL tables, using sqlite3.

In [4]:
## Create the food table with the following commands
# Create table statement
createFoodTable = '''
  CREATE TABLE IF NOT EXISTS food (
          fdc_id INT PRIMARY KEY,
          data_type VARCHAR(30),
          description VARCHAR(40),
          food_category_id INT NOT NULL,
          publication_date DATE
  );
'''
# The csv holding the food information
foodCSV = 'food.csv'
# The insertion statement
foodInsertTable = "INSERT INTO food VALUES (?, ?, ?, ?, ?)"
# Drop the table if it exists
dropTable('food')
# Now, use our generate function to create the table
generateDB(createFoodTable, foodCSV, foodInsertTable)

In [5]:
# Now, let's look at the unfiltered database
print('Total Items: ', runQuery('SELECT COUNT(*) FROM food;')[0][0])
print('First 5 items:')
result = runQuery('SELECT * FROM food LIMIT 5;')
formatted_row = '{:<10} {:<20} {:<30} {:<20} {:<10}'
print(formatted_row.format("fdc_id", "data_type", "description", "food_category_id", "publication_date"))
for Row in result:
    print(formatted_row.format(*Row))

Total Items:  60432
First 5 items:
fdc_id     data_type            description                    food_category_id     publication_date
319874     sample_food          HUMMUS, SABRA CLASSIC          16                   2019-04-01
319875     market_acquisition   HUMMUS, SABRA CLASSIC          16                   2019-04-01
319876     market_acquisition   HUMMUS, SABRA CLASSIC          16                   2019-04-01
319877     sub_sample_food      Hummus                         16                   2019-04-01
319878     sub_sample_food      Hummus                         16                   2019-04-01


As you can see, the data is a bit messy. Even in this small sample, we can see hummus multiple times, which for some people is simply too much hummus! This dataset is split into different types of food data types. Let's look at 'foundation_food', which comprises core foundation food items from which other foodstuffs are made.

In [6]:
print('Total Items: ', runQuery('SELECT COUNT(*) FROM (SELECT * FROM food WHERE data_type = "foundation_food");')[0][0])
print('First 5 items:')
result = runQuery('SELECT * FROM food WHERE data_type = "foundation_food" LIMIT 5;')
formatted_row = '{:<10} {:<18} {:<75} {:<20} {:<10}'
print(formatted_row.format("fdc_id", "data_type", "description", "food_category_id", "publication_date"))
for Row in result:
    print(formatted_row.format(*Row))

Total Items:  336
First 5 items:
fdc_id     data_type          description                                                                 food_category_id     publication_date
321358     foundation_food    Hummus, commercial                                                          16                   2019-04-01
321359     foundation_food    Milk, reduced fat, fluid, 2% milkfat, with added vitamin A and vitamin D    1                    2019-04-01
321360     foundation_food    Tomatoes, grape, raw                                                        11                   2019-04-01
321505     foundation_food    Salt, table, iodized                                                        2                    2019-04-01
321611     foundation_food    Beans, snap, green, canned, regular pack, drained solids                    11                   2019-04-01


<hr>

### More Tables

This project focuses on data relationship between multiple tables in the FDA database to draw insights into food data that might be relevant for consumers. Here, we will use our previous functions to create the following tables:
- nutrient table
- portion Table
- nutrient_conversion_factor Table
- calorie_conversion_factor Table

Each of these will allow us, through various queries, to draw conclusions and relationships between each selected food item.

In [7]:
## Create the nutrient table
## Note the fdc_id Foreign Key that aligns with our food table. The index is provided by a nutrient id
createNutrientTable = '''
  CREATE TABLE IF NOT EXISTS nutrient (
          id INT PRIMARY KEY,
          fdc_id INT,
          nutrient_id INT,
          amount REAL,
          data_points INT,
          derivation_id INT,
          min INT,
          max INT,
          median INT,
          footnote VARCHAR(40),
          min_year_acquired DATE,
          FOREIGN KEY (fdc_id) REFERENCES food(fdc_id)
  );
'''
nutrientCSV = 'food_nutrient.csv'
nutrientInsertTable = "INSERT INTO nutrient VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
dropTable('nutrient')
generateDB(createNutrientTable, nutrientCSV, nutrientInsertTable)
print('Total Items: ', runQuery('SELECT COUNT(*) FROM nutrient;')[0][0])
print('First 5 items:')
result = runQuery('SELECT * FROM nutrient LIMIT 5;')
formatted_row = '{:<10} {:<10} {:<15} {:<10} {:<15} {:<10}'
print(formatted_row.format("id", "fdc_id", "nutrient_id", "amount", "data_points", "derivation_id"))
for Row in result:
    print(formatted_row.format(*Row))

Total Items:  140824
First 5 items:
id         fdc_id     nutrient_id     amount     data_points     derivation_id
2201847    319877     1051            56.3       1               1         
2201845    319877     1002            1.28       1               1         
2201846    319877     1004            19.0       1               1         
2201844    319877     1007            1.98       1               1         
2201852    319878     1091            188.0      1               1         


In [8]:
# portion table
## Note the fdc_id Foreign Key that aligns with our food table. The index is provided by a portion id
createPortionTable = '''
  CREATE TABLE IF NOT EXISTS portion (
          id INT PRIMARY KEY,
          fdc_id INT,
          seq_num INT,
          amount REAL NOT NULL,
          measure_unit_id INT NOT NULL,
          portion_description VARCHAR(10),
          modifier VARCHAR(50),
          gram_weight REAL NOT NULL,
          data_points INT,
          footnote VARCHAR(40),
          min_year_acquired DATE,
          FOREIGN KEY (fdc_id) REFERENCES food(fdc_id)
  );
'''
portionCSV = 'food_portion.csv'
portionInsertTable = "INSERT INTO portion VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
dropTable('portion')
generateDB(createPortionTable, portionCSV, portionInsertTable)
print('Total Items: ', runQuery('SELECT COUNT(*) FROM portion;')[0][0])
print('First 5 items:')
result = runQuery('SELECT * FROM portion LIMIT 5;')
formatted_row = '{:<10} {:<10} {:<10} {:<10} {:<20} {:<20} {:<10} {:<20}'
print(formatted_row.format("id", "fdc_id", "seq_num", "amount", "measure_unit_id", "portion_description", "modifier", "gram_weight"))
for Row in result:
    print(formatted_row.format(*Row))

Total Items:  10678
First 5 items:
id         fdc_id     seq_num    amount     measure_unit_id      portion_description  modifier   gram_weight         
118702     319875                2.0        1001                                                 35.8                
118703     319880                2.0        1001                                                 36.2                
118704     319881                2.0        1001                                                 33.4                
118705     319886                2.0        1001                                                 33.2                
118706     319889                2.0        1001                                                 34.6                


In [9]:
# nutrient_conversion_factor Table
## Note the fdc_id Foreign Key that aligns with our food table. The index is provided by a nutrient_conversion_factor id
## Note that not all fdc_ids here exist in food, so maybe create a tirgger for that
createNutrientConversionFactorTable = '''
  CREATE TABLE IF NOT EXISTS nutrient_conversion_factor (
          id INT PRIMARY KEY,
          fdc_id NOT NULL
  );
'''
nutrientConversionFactorCSV = 'food_nutrient_conversion_factor.csv'
nutrientConversionFactorInsertTable = "INSERT INTO nutrient_conversion_factor VALUES (?, ?)"
dropTable('nutrient_conversion_factor')
generateDB(createNutrientConversionFactorTable, nutrientConversionFactorCSV, nutrientConversionFactorInsertTable)
print('Total Items: ', runQuery('SELECT COUNT(*) FROM nutrient_conversion_factor;')[0][0])
print('First 5 items:')
result = runQuery('SELECT * FROM nutrient_conversion_factor LIMIT 5;')
formatted_row = '{:<10} {:<10}'
print(formatted_row.format("id", "fdc_id"))
for Row in result:
    print(formatted_row.format(*Row))

Total Items:  11326
First 5 items:
id         fdc_id    
22721      746952    
22722      747429    
22723      747693    
22724      749420    
22725      789890    


In [10]:
# calorie_conversion_factor Table
## Note the nutrient_conversion_factor Foreign Key that aligns with our nutrient_conversion_factor table.
## The index is provided by a nutrient id
createCalorieConversionFactorTable = '''
  CREATE TABLE IF NOT EXISTS calorie_conversion_factor (
      food_nutrient_conversion_factor_id INT PRIMARY KEY,
      protein_value REAL NOT NULL,
      fat_value REAL NOT NULL,
      carbohydrate_value REAL NOT NULL,
      FOREIGN KEY (food_nutrient_conversion_factor_id) REFERENCES nutrient_conversion_factor(id)
  );
'''
calorieConversionFactorCSV = 'food_calorie_conversion_factor.csv'
calorieConversionFactorInsertTable = "INSERT INTO calorie_conversion_factor VALUES (?, ?, ?, ?)"
dropTable('calorie_conversion_factor')
generateDB(createCalorieConversionFactorTable, calorieConversionFactorCSV, calorieConversionFactorInsertTable)
print('Total Items: ', runQuery('SELECT COUNT(*) FROM calorie_conversion_factor;')[0][0])
print('First 5 items:')
result = runQuery('SELECT * FROM calorie_conversion_factor LIMIT 5;')
formatted_row = '{:<40} {:<15} {:<15} {:<15}'
print(formatted_row.format("food_nutrient_conversion_factor_id", "protein_value", "fat_value", "carbohydrate_value"))
for Row in result:
    print(formatted_row.format(*Row))

Total Items:  290
First 5 items:
food_nutrient_conversion_factor_id       protein_value   fat_value       carbohydrate_value
22503                                    3.47            8.37            4.07           
22505                                    4.27            8.79            3.87           
22507                                    2.44            8.37            3.57           
22510                                    2.44            8.37            3.57           
22512                                    2.44            8.37            3.57           


In [11]:
# Nutrient Definitions Table
# This will display the types of nutrients by nutrient id
createNutrientDefTable = '''
    CREATE TABLE IF NOT EXISTS nutrient_definitions (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        unit_name VARCHAR(20),
        nutrient_nbr INT,
        rank REAL
    );
'''
nutrientDefCSV = 'nutrient.csv'
nutrientDefInsertTable = "INSERT INTO nutrient_definitions VALUES (?, ?, ?, ?, ?)"
dropTable('nutrient_definitions')
generateDB(createNutrientDefTable, nutrientDefCSV, nutrientDefInsertTable)
print('Total Items: ', runQuery('SELECT COUNT(*) FROM nutrient_definitions;')[0][0])
print('First 5 items:')
result = runQuery('SELECT * FROM nutrient_definitions LIMIT 5;')
formatted_row = '{:<10} {:<40} {:<15} {:<15} {:<15}'
print(formatted_row.format("id", "name", "unit_name", "nutrient_nbr", "rank"))
for Row in result:
    print(formatted_row.format(*Row))

Total Items:  477
First 5 items:
id         name                                     unit_name       nutrient_nbr    rank           
2047       Energy (Atwater General Factors)         KCAL            957             280.0          
2048       Energy (Atwater Specific Factors)        KCAL            958             290.0          
1001       Solids                                   G               201             200.0          
1002       Nitrogen                                 G               202             500.0          
1003       Protein                                  G               203             600.0          


<hr>

### Advanced Queries

Now that our tables are created, we can start making some unique queries. In the first query, we will be using a CTEs and various types of apples to determine if there are any nutritional differences between the various types, as determined by the FDA. In this first query, we are just going to print out a list of the macronutrients associated with apples grouped by description, which yields five different apples in the "foundational_food" data_type:
- Red Delicious
- Fuji
- Gala
- Granny Smith
- Honeycrisp

In [12]:
## Simple query to get our base table of apples. We're also going to retrieve the portion size
## to see if there's a difference in the weight of each apple type
result = runQuery('''
    SELECT apples.fdc_id, description, gram_weight
    FROM (SELECT * FROM food
    WHERE description LIKE '%APPLES,%'
    and data_type = "foundation_food") as apples, portion
    GROUP BY description
''')
formatted_row = '{:<10} {:<40} {:<40}'
print(formatted_row.format("FDC ID", "Description", "Grams"))
for Row in result:
    print(formatted_row.format(*Row))

FDC ID     Description                              Grams                                   
1105897    Apples, fuji, with skin, raw             35.8                                    
1105781    Apples, gala, with skin, raw             35.8                                    
1105664    Apples, granny smith, with skin, raw     35.8                                    
1105547    Apples, honeycrisp, with skin, raw       35.8                                    
1105430    Apples, red delicious, with skin, raw    35.8                                    


In [13]:
## Query to determine the nutritional conversion of different apples.
## This joins the food, nutrient_conversion_factor, and calorie_conversion_factor tables

## First, because we are going to look at different nutrient_ids, lets make a function so we can run this query multiple times
def appleQuery(andStatement):
    return runQuery('''
        WITH 
        ids AS (
            WITH apples AS (
                SELECT *
                FROM food
                WHERE description LIKE '%APPLES,%'
                GROUP BY description
            )
            SELECT NC.fdc_id, A.description as apple
            FROM apples AS A, nutrient_conversion_factor AS NC, calorie_conversion_factor as C
            WHERE A.fdc_id = NC.fdc_id AND C.food_nutrient_conversion_factor_id = NC.id
        ),
        nutrients AS (
            SELECT * FROM nutrient, ids WHERE nutrient.fdc_id = ids.fdc_id
        )
        SELECT nutrient_id, apple, amount, unit_name, name FROM nutrients, nutrient_definitions
            WHERE nutrients.nutrient_id = nutrient_definitions.id
    '''+andStatement+';')

# Now just check for nutrient_ids 1003-1005, which are protein, fat, and carbs
result = appleQuery('AND nutrient_id IN (1003,1004,1005)')
formatted_row = '{:<10} {:<40} {:<10} {:<10} {:<10}'
print(formatted_row.format("Nutrient", "Apple Type", "Amount", "Units", "Nutrient"))
for Row in result:
    print(formatted_row.format(*Row))

Nutrient   Apple Type                               Amount     Units      Nutrient  
1003       Apples, red delicious, with skin, raw    0.19       G          Protein   
1003       Apples, fuji, with skin, raw             0.15       G          Protein   
1003       Apples, gala, with skin, raw             0.13       G          Protein   
1003       Apples, granny smith, with skin, raw     0.27       G          Protein   
1003       Apples, honeycrisp, with skin, raw       0.1        G          Protein   
1004       Apples, red delicious, with skin, raw    0.21       G          Total lipid (fat)
1004       Apples, fuji, with skin, raw             0.16       G          Total lipid (fat)
1004       Apples, gala, with skin, raw             0.15       G          Total lipid (fat)
1004       Apples, granny smith, with skin, raw     0.14       G          Total lipid (fat)
1004       Apples, honeycrisp, with skin, raw       0.1        G          Total lipid (fat)
1005       Apples, red delicio

<hr>

#### Sugars

Already we can see some interesting things. While close in values, we can immediately see that Granny Smith apples are much higher in protein than the other apples, whereas Red Delicious leads the pack in fat. Fuji then blows away the competition (relatively) in terms of carbs. We can adjust this query slightly by searching for the total sugars in each apple and see if that's why Fuji apples are so high in carbs.
<hr>

In [14]:
# Now just check for nutrient_id 1063, which is total sugars
result = appleQuery('AND nutrient_id = 1063 ORDER BY amount DESC')
formatted_row = '{:<10} {:<40} {:<10} {:<10} {:<10}'
print(formatted_row.format("Nutrient", "Apple Type", "Amount", "Units", "Nutrient"))
for Row in result:
    print(formatted_row.format(*Row))

Nutrient   Apple Type                               Amount     Units      Nutrient  
1063       Apples, fuji, with skin, raw             12.6       G          Sugars, Total
1063       Apples, honeycrisp, with skin, raw       11.7       G          Sugars, Total
1063       Apples, gala, with skin, raw             11.3       G          Sugars, Total
1063       Apples, red delicious, with skin, raw    11.1       G          Sugars, Total
1063       Apples, granny smith, with skin, raw     10.5       G          Sugars, Total


<hr>

#### Fructose
Aha! Fuji apples are indeed the highest in total sugar count, just as we suspected. We can now search based on any nutrient ID that appears in the total list. However, a personal favorite apple type of mine is the Honeycrisp. Honeycrisp leads the pack in fructose, so it's possible that's why I like that type of apple so much:
<hr>

In [15]:
# Now just check for nutrient_id 1012, which is fructose
result = appleQuery('AND nutrient_id = 1012 ORDER BY amount DESC')
formatted_row = '{:<10} {:<40} {:<10} {:<10} {:<10}'
print(formatted_row.format("Nutrient", "Apple Type", "Amount", "Units", "Nutrient"))
for Row in result:
    print(formatted_row.format(*Row))

Nutrient   Apple Type                               Amount     Units      Nutrient  
1012       Apples, honeycrisp, with skin, raw       8.56       G          Fructose  
1012       Apples, fuji, with skin, raw             8.4        G          Fructose  
1012       Apples, gala, with skin, raw             7.76       G          Fructose  
1012       Apples, red delicious, with skin, raw    7.56       G          Fructose  
1012       Apples, granny smith, with skin, raw     6.76       G          Fructose  


<hr>

#### Calcium
And one final interesting note to display the capabilities of this query is looking at a microingredient that many people find important, calcium. Surprisingly, our pack leaders lag behing the Gala apple, which has a full mg more calcium than its closest competitor! I'll have to get my calcium elsewhere, as honeycrisp is bringing up the rear.

<hr>

In [16]:
# Now just check for nutrient_id 1087, which is calcium
result = appleQuery('AND nutrient_id = 1087 ORDER BY amount DESC')
formatted_row = '{:<10} {:<40} {:<10} {:<10} {:<10}'
print(formatted_row.format("Nutrient", "Apple Type", "Amount", "Units", "Nutrient"))
for Row in result:
    print(formatted_row.format(*Row))

Nutrient   Apple Type                               Amount     Units      Nutrient  
1087       Apples, gala, with skin, raw             7.0        MG         Calcium, Ca
1087       Apples, fuji, with skin, raw             6.0        MG         Calcium, Ca
1087       Apples, granny smith, with skin, raw     5.0        MG         Calcium, Ca
1087       Apples, red delicious, with skin, raw    5.0        MG         Calcium, Ca
1087       Apples, honeycrisp, with skin, raw       4.0        MG         Calcium, Ca


<hr>

### Protein

Protein is the name of the game in many diet-concious people these days. With that in mind, a use of this database could be to determine the highest protein content per weight in the FDA database. Per the [FDA's website](https://www.fao.org/3/Y5022E/y5022e04.htm), different types of proteins result in different multipliers (i.e. caloric conversion factors) for each food. So, let's say we wanted to do an advanced search to identify the highest protein multiplier in the FDA database, with the lowest carbohydrate multiplier. We conduct this query below, using three tables, a grouping, and an aggregator on the protein_value.
<hr>

In [17]:
## Query to determine the highest protein value multiplier amongst the foundational_foods
result = runQuery('''
    WITH foods AS (
        SELECT *
        FROM food
        WHERE data_type = "foundation_food"
    )
    SELECT F.fdc_id as 'FDC ID', F.description as 'Food', MAX(C.protein_value), C.fat_value, MIN(C.carbohydrate_value)
    FROM foods AS F, nutrient_conversion_factor AS N, calorie_conversion_factor as C
    WHERE F.fdc_id = N.fdc_id AND C.food_nutrient_conversion_factor_id = N.id
    GROUP BY food_category_id
    ORDER BY C.protein_value DESC LIMIT 10;
''')
formatted_row = '{:<10} {:<100} {:>10} {:>10} {:>10}'
print(formatted_row.format("FDC ID", "Food", "Protein", "Fat", "Carbs"))
for Row in result:
    print(formatted_row.format(*Row))

FDC ID     Food                                                                                                    Protein        Fat      Carbs
323604     Egg, whole, raw, frozen, pasteurized                                                                       4.36       9.02       3.68
333374     Fish, haddock, raw                                                                                         4.27       9.02       3.87
334849     Beef, loin, top loin steak, boneless, lip-on, separable lean only, trimmed to 1/8" fat, choice, raw        4.27       9.02       3.87
746952     Ham, sliced, restaurant                                                                                    4.27       9.02       3.87
323121     Frankfurter, beef, unheated                                                                                4.27       9.02       3.87
330869     Turkey, ground, 93% lean, 7% fat, pan-broiled crumbles                                                     4.27       9

<hr>
The humble egg edges out all of it's competitors. While the difference may be small, remember that the caloric_conversion_factor is a multiplier for the amount of protein, fat, and carbs inside the specific food item to calculate the true caloric value of the food. Now, let's take a look at the various macro and micro ingredients in side the egg by searching for it's fdc_id in the join between nutrient and nutrient definitions.
<hr>

In [18]:
## Ok, we can see that eggs have the highest protein multiplier. Let's see what other nutrients are present in eggs
result = runQuery('''
    WITH 
    N AS (
        SELECT * FROM nutrient WHERE fdc_id = 323604
    ),
    nutrients AS (
        SELECT name, unit_name, amount, fdc_id FROM nutrient_definitions JOIN N WHERE N.nutrient_id = nutrient_definitions.id
    )
    SELECT * from nutrients;
''')
formatted_row = '{:<50} {:<10} {:>10}'
print(formatted_row.format("Nutrient", "Unit", "Amount"))
for Row in result:
    print(formatted_row.format(*Row))

Nutrient                                           Unit           Amount
Carbohydrate, by difference                        G                0.91
Energy                                             KCAL            150.0
Iron, Fe                                           MG               1.77
25-hydroxycholecalciferol                          UG                0.6
Water                                              G                75.4
Sodium, Na                                         MG              121.0
Ash                                                G                1.16
Vitamin D (D2 + D3), International Units           IU               91.0
Protein                                            G                12.3
Iodine, I                                          UG               61.6
Zinc, Zn                                           MG                1.2
Manganese, Mn                                      MG                0.0
Potassium, K                                       

<hr>
As you can see, because we can query for a large number of micronutrients, not only could we establish queries for things like calories, protein, carbs, and fats, we could also optimize a user's dietary requirements down to micronutrients like calcium or vitamin D! However, we are merely interested in using the protein, fat, and carb conversions to calculate the caloric value of an egg. While we could simply reference the "Energy" item in the nutrients list, it's a bit more interesting to work through the query to calculate each value for an egg. The below query does the following:

* Joins 4 tables on various indexes
* Strips out the protein, fat, and carb multipliers and macronutrient values
* Multiplies the values
* Sums the result
<hr>

In [19]:
## Ok, now that we've found the egg has the highest protein conversion factor, let's calculate the caloric energy in an egg!
result = runQuery('''
    WITH 
    N AS (
        SELECT * FROM nutrient WHERE fdc_id = 323604
    ),
    nutrients AS (
        SELECT name, unit_name, amount, fdc_id FROM nutrient_definitions JOIN N WHERE N.nutrient_id = nutrient_definitions.id
    ),
    cals AS (
        SELECT * FROM nutrient_conversion_factor as NC, calorie_conversion_factor as C
        WHERE NC.fdc_id = '323604' AND C.food_nutrient_conversion_factor_id = NC.id
    ),
    macros AS (
        SELECT name, amount, protein_value, fat_value, carbohydrate_value FROM cals, nutrients where cals.fdc_id = nutrients.fdc_id
        AND name in ("Protein", "Total lipid (fat)", "Carbohydrate, by difference")
    ),
    egg_calories AS (
        SELECT * FROM 
            (SELECT amount * protein_value as proteins FROM macros WHERE name = "Protein"),
            (SELECT amount * fat_value as fats FROM macros WHERE name = "Total lipid (fat)"),
            (SELECT amount * carbohydrate_value as carbs FROM macros WHERE name = "Carbohydrate, by difference")
    )
    SELECT proteins + fats + carbs FROM egg_calories;
''')
print('Egg calculated KCALs: ',result[0][0])


Egg calculated KCALs:  149.88280000000003


<hr>

### Inserting New Data

As we saw when creating the food table, the descriptions are often duplicated, and relatively unhelpful. So, we are going to create a trigger for our database enforcing a unique description based on the LIKE pattern matching. This will ensure we don't have any more duplicates such as various hummus types and so forth. We will also raise a custom integrity error so the user knows what the issue is.

In [20]:
## Trigger to enforce only inserting a food description that doesn't already exist in the LIKE pattern match
## Note: We do this as a BEFORE INSERT statement
conn = sqlite3.connect(database)
c = conn.cursor()
c.execute('DROP TRIGGER IF EXISTS no_repeat_descriptions')
c.execute('''
CREATE TRIGGER IF NOT EXISTS no_repeat_descriptions
BEFORE INSERT ON food
FOR EACH ROW
WHEN (SELECT COUNT(*) FROM food WHERE description LIKE '%' || new.description || '%') > 0
BEGIN
    SELECT RAISE(FAIL,'Description must be unique');
END
''')
conn.commit()
conn.close()

<hr>
Great, so now let's test this to make sure we can't insert yet another hummus.
<hr>

In [21]:
## Lets test the trigger by attempting to insert yet another hummus. This should fail!
conn = sqlite3.connect(database)
c = conn.cursor()
try:
    c.execute('''
        INSERT INTO food 
        VALUES (
            1,
            "sample_food", 
            "hummus",
            30,
            '2023-12-10'
        );''').fetchall()
except Exception as error:
    print('SQL ERROR: ',error)
conn.commit()
conn.close()

SQL ERROR:  Description must be unique


<hr>
Excellent, we've stopped the unblocked entry of multiple hummus items. However, does this let us insert new items? Strangely, this selection of the FDA database doesn't have an entry for chocolate, so we will test using that. First, I'll prove that no chocolate entry exists:
<hr>

In [22]:
runQuery('SELECT * FROM food WHERE description LIKE "%chocolate%"')

[]

<hr>
Now let's correct this eggregious oversight by adding an entry for chocolate
<hr>

In [23]:
## Insert some chocolate
conn = sqlite3.connect(database)
c = conn.cursor()
# Get the max fdc_id number and add one
chocolate_fdc_id = c.execute('''
    SELECT MAX(fdc_id) FROM food;
''').fetchall()[0][0] + 1
insertStatement = 'INSERT INTO food VALUES ('+str(chocolate_fdc_id)+',"foundation_food", "chocolate", 30,"2023-12-10");'
c.execute(insertStatement)
conn.commit()
conn.close()
runQuery('SELECT * FROM food WHERE description LIKE "%chocolate%"')

[(2648874, 'foundation_food', 'chocolate', 30, '2023-12-10')]

<hr>
Ok, we've corrected that grave error, but we don't have any other table entries for chocolate. Let's insert a few more entries for chocolate into the other tables, like calories. First, we need to create a conversion factor entry.
<hr>

In [24]:
## Insert chocolate into the nutrient_conversion_factor table
conn = sqlite3.connect(database)
c = conn.cursor()
# Get the max id number and add one
chocolate_nutrient_conversion_factor = c.execute('''
    SELECT MAX(id) FROM nutrient_conversion_factor;
''').fetchall()[0][0] + 1
c.execute(('INSERT INTO nutrient_conversion_factor VALUES ('
           +str(chocolate_nutrient_conversion_factor)
           +','
           +str(chocolate_fdc_id)+');'))
conn.commit()
conn.close()
result = runQuery(('SELECT * FROM nutrient_conversion_factor WHERE id = '
          +str(chocolate_nutrient_conversion_factor)
          +';'))
formatted_row = '{:<10} {:<10}'
print(formatted_row.format("id", "fdc_id"))
for Row in result:
    print(formatted_row.format(*Row))

id         fdc_id    
23078      2648874   


<hr>
Now that we have a conversion factor entry, we can add an entry to the calorie conversion factor table. We will simply use the standard Atwater General Factor System values of 4.0, 9.0, and 4.0 for protein, fat, and carbs respectively, as the specific factors are beyond the scope of this course.
<hr>

In [25]:
## Insert chocolate into the calorie_conversion_factor table
conn = sqlite3.connect(database)
c = conn.cursor()
c.execute(('INSERT INTO calorie_conversion_factor VALUES ('
           +str(chocolate_nutrient_conversion_factor)
           +',4.0, 9.0, 4.0'
           +');'))
conn.commit()
conn.close()
result = runQuery(('SELECT * FROM calorie_conversion_factor WHERE food_nutrient_conversion_factor_id = '
         +str(chocolate_nutrient_conversion_factor)+';'))
formatted_row = '{:<40} {:<15} {:<15} {:<15}'
print(formatted_row.format("food_nutrient_conversion_factor_id", "protein_value", "fat_value", "carbohydrate_value"))
for Row in result:
    print(formatted_row.format(*Row))

food_nutrient_conversion_factor_id       protein_value   fat_value       carbohydrate_value
23078                                    4.0             9.0             4.0            


<hr>
Now let's assume that we have new data with more specific protein/fat/carb values. Let's update the database to reflect our new values.
<hr>

In [26]:
## Update our conversion factors for chocolate
conn = sqlite3.connect(database)
c = conn.cursor()
c.execute(('UPDATE calorie_conversion_factor SET'
           + ' protein_value = protein_value - 0.5,'
           + ' fat_value = fat_value + 1.5,'
           + ' carbohydrate_value = 17.0\n'
           + 'WHERE food_nutrient_conversion_factor_id = '
           +str(chocolate_nutrient_conversion_factor)
           +';'))
conn.commit()
conn.close()
result = runQuery(('SELECT * FROM calorie_conversion_factor WHERE food_nutrient_conversion_factor_id = '
         +str(chocolate_nutrient_conversion_factor)+';'))
formatted_row = '{:<40} {:<15} {:<15} {:<15}'
print(formatted_row.format("food_nutrient_conversion_factor_id", "protein_value", "fat_value", "carbohydrate_value"))
for Row in result:
    print(formatted_row.format(*Row))

food_nutrient_conversion_factor_id       protein_value   fat_value       carbohydrate_value
23078                                    3.5             10.5            17.0           


<hr>
Ok great, insertion and update worked. But, there was probably a good reason that chocolate isn't in our database. Maybe it's in another location in a different csv. Either way, the step below will show how we can remove old or unwanted data from this database. Remember that these items all have foreign keys. calorie_conversion_factor is using the nutrient_conversion_factor_id as the foreign key, and nutrient_conversion_factor is using the fdc_id as the foreign key. There are two ways we can delete this entry. We can go backwards and remove the calorie_conversion_factor entry first, moving up to the food table, add a new trigger, or we can UPDATE our table to include an ON CASCADE.
<hr>

In [27]:
# First, let's try to remove chocolate from the calorie conversion table. It should fail.
conn = sqlite3.connect(database)
c = conn.cursor()
c.execute('PRAGMA foreign_keys=on;')
try:
    c.execute(('DELETE FROM nutrient_conversion_factor WHERE id = '
               +str(chocolate_nutrient_conversion_factor)
               +';'))
except Exception as error:
    print(error)
conn.commit()
conn.close()
result = runQuery(('SELECT * FROM nutrient_conversion_factor WHERE id = '
               +str(chocolate_nutrient_conversion_factor)
               +';'))
formatted_row = '{:<10} {:<10}'
print(formatted_row.format("id", "fdc_id"))
for Row in result:
    print(formatted_row.format(*Row))

FOREIGN KEY constraint failed
id         fdc_id    
23078      2648874   


<hr>
As we expected, that shouldn't have worked. As mentioned above, we COULD alter our table to include the ON CASCADE rule, but instead we are going to add a trigger to handle this cascading deletion for us.
<hr>

In [28]:
## Trigger to allow for cascade deletion from nutrient_conversion_factor
conn = sqlite3.connect(database)
c = conn.cursor()
c.execute('PRAGMA foreign_keys=on;')
c.execute('DROP TRIGGER IF EXISTS delete_nutrient_conversion_factor')
c.execute('''
CREATE TRIGGER IF NOT EXISTS delete_nutrient_conversion_factor
BEFORE DELETE ON nutrient_conversion_factor
FOR EACH ROW
BEGIN
    DELETE FROM calorie_conversion_factor WHERE food_nutrient_conversion_factor_id = old.id;
END
''')
conn.commit()
conn.close()

In [29]:
conn = sqlite3.connect(database)
c = conn.cursor()
c.execute('PRAGMA foreign_keys=on;')
c.execute(('DELETE FROM nutrient_conversion_factor WHERE id = '
           +str(chocolate_nutrient_conversion_factor)
           +';'))
conn.commit()
conn.close()
nutrientConversionFactor = runQuery(('SELECT * FROM nutrient_conversion_factor WHERE id = '
               +str(chocolate_nutrient_conversion_factor)
               +';'))
calorieConversionFactor = runQuery(('SELECT * FROM calorie_conversion_factor WHERE food_nutrient_conversion_factor_id = '
               +str(chocolate_nutrient_conversion_factor)
               +';'))
print('Nutrient Conversion Factor: ', nutrientConversionFactor)
print('Calorie Conversion Factor: ', calorieConversionFactor)

Nutrient Conversion Factor:  []
Calorie Conversion Factor:  []


<hr>

## Summary

It almost goes without saying the value a database like this might have for the average consumer who is attempting to better understand their diet, caloric intake, and nutritional requirements. Being able to quickly and efficiently compare different food types, down to different types of apples, and assess which type best fits the dietary needs of the consumer would greatly enable more intelligent shopping. A secondary effect might be decreased food waste, more savings, and so on.

On top of the user benefits of a more comprehensive application, this project also greatly contributed to my understanding of database creation and management, in line with the goals of this project laid out in the submission. During the creation of this project, I was able to:

- Better understand how to implement multiple relations across large datasets.
- Implement a real-world use case dataset solution for an existing problem set.
- Understand how to seed a database from an existing CSV.
- Implement and use a trigger in an actual use case.
- Better understand the basic database concepts taught during this course.
- Gain an appreciation for database projects currently in production (Facebook, Amazon, etc.).
- Specifically, gain a better working knowledge of the FDA databases and derive meaningful insights from correlating data across multiple tables.

<hr>